In [2]:
pip install statsmodels

  Using cached statsmodels-0.14.6-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 53.2 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Data Analysis and Statistical Modeling for Bakery Sales Prediction
# ---------------------------------------------------------------
# This script imports necessary libraries and loads data for analyzing 
# the relationship between weather conditions and bakery sales.

# Import der Umsatz- und der Wetter-Daten
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
# Note: Required for regression analysis, time series analysis, etc.
import statsmodels.api as sm

weather = "/workspaces/bakery_prediction/0_DataPreparation/wetter.csv"
weather = pd.read_csv(weather)
print(weather)

test = "/workspaces/bakery_prediction/0_DataPreparation/test.csv"
test = pd.read_csv(test)
print(test)

train = "/workspaces/bakery_prediction/0_DataPreparation/train.csv"
train = pd.read_csv(train)
print(train)

           Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
0     2012-01-01         8.0      9.8250                   14        58.0
1     2012-01-02         7.0      7.4375                   12         NaN
2     2012-01-03         8.0      5.5375                   18        63.0
3     2012-01-04         4.0      5.6875                   19        80.0
4     2012-01-05         6.0      5.3000                   23        80.0
...          ...         ...         ...                  ...         ...
2596  2019-07-28         3.0     23.3500                   14         5.0
2597  2019-07-29         6.0     25.2500                    7        61.0
2598  2019-07-30         7.0     20.7375                    8        61.0
2599  2019-07-31         6.0     20.4500                    7        61.0
2600  2019-08-01         5.0     21.0625                    9        61.0

[2601 rows x 5 columns]
           id       Datum  Warengruppe
0     1808011  2018-08-01            1
1     180